In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras import models 
from tensorflow.keras import layers 
from tensorflow.keras import optimizers 
from tensorflow.keras import regularizers 
from tensorflow.keras.layers import Embedding, Flatten, Dense 
import tensorflow as tf
import keras
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import callbacks
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from time import time
import re

from copy import deepcopy
from keras.layers import *
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.metrics import *
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
reviews=pd.read_csv("FacebookBig.csv").loc[:, ['content', 'score']]
reviews

,content,score
0,Does every damn video I watch has 16 seconds o...,1
1,Good,5
2,Hgb,5
3,Oo his is so amazing app,5
4,My Facebook marketplace is not work,1
...,...,...
300595,Nc,5
300596,Good,5
300597,👍,5
300598,Good,5


In [3]:
reviews['score'].value_counts()

5    166722
1     77208
4     21881
3     18124
2     16665
Name: score, dtype: int64

In [4]:
ENGLISH_CHARACTERS = 'abcdefgjihklmnopqrstuvwxyz'
ENGLISH_CHARACTERS += ENGLISH_CHARACTERS.upper() + ' '
ENGLISH_CHARACTERS = set(ENGLISH_CHARACTERS)
#!?.,\'"
ENGLISH_STOPWORDS = set(stopwords.words('english'))

# for word in "wouldn't doesn't couldn't shouldn't didn't haven't aren't mightn't wasn't weren't hasn't hadn't".split():
#     ENGLISH_STOPWORDS.remove(word)

ENGLISH_STOPWORDS = set()
    
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()
    
print(ENGLISH_CHARACTERS)
print()
print(ENGLISH_STOPWORDS)
print()
print(stemmer)
print(lemma)

{'n', 'F', 'M', 'Q', 's', 't', 'l', 'Y', 'G', 'O', 'C', 'o', 'V', 'P', 'g', 'X', 'f', 'h', 'B', 'w', ' ', 'c', 'J', 'u', 'j', 'K', 'r', 'W', 'i', 'y', 'm', 'Z', 'k', 'b', 'A', 'e', 'z', 'p', 'H', 'R', 'N', 'D', 'q', 'S', 'L', 'T', 'x', 'E', 'd', 'I', 'v', 'U', 'a'}

set()

<PorterStemmer>
<WordNetLemmatizer>


In [5]:
'''
non-english characters
    if too many, we remove it

lowercase
stopword removal
stemming
'''


def clean(text):
    text = text.lower()
    
    # removal of non-english characters
    new = ''
    for ch in text:
        if ch in ENGLISH_CHARACTERS:
            new += ch
    text = new
    
    # if len(text) < 12:
    #     return ''
    
    tokens = word_tokenize(text)
    tokens = [lemma.lemmatize(t) for t in tokens if t not in ENGLISH_STOPWORDS]
    return ' '.join(tokens)
    

clean('Today my whatsapp account has been blocked%%%%%%%%%')

'today my whatsapp account ha been blocked'

In [85]:
reviews = pd.read_csv("Amogus.csv").loc[:, ['content', 'score']]
reviews['content'] = [clean(str(c)) for c in reviews['content']]
reviews = reviews[[len(i) >=20 for i in reviews['content']]]

reviews.head(20)

,content,score
0,it is so asome and also it is interesting,5
4,do not install the game,1
5,everything will be ok,4
6,cosmicubes not working for me,3
7,dear developer nice game but people play this ...,1
9,i cant play with my friend or cousin when i jo...,1
11,among u is a great game but it ha some downfal...,2
13,whenever i become an imposter i always get dis...,2
14,worst game in the world ever,1
15,whn the google play is sus,5


In [86]:
reviews.score.value_counts()

5    22266
1    21669
4     7767
3     6724
2     5358
Name: score, dtype: int64

In [87]:
dfs = []
nums = []
for score in [1,2,3,4,5]:
    temp = reviews[reviews['score']==score]
    dfs.append(temp)
    nums.append(temp.shape[0])
    
dfs = [df.sample(min(nums)) for df in dfs]

data = pd.concat(dfs, axis=0).sample(frac=1)
display(data)

data.score.value_counts()

,content,score
93716,doesnt work to join other people,2
10703,ist nice but it whould be better if we can wri...,4
76487,the new stuff is dirt,2
7838,this game used to be fun and it still is my on...,3
53927,this game is very cool try the game please thi...,5
...,...,...
62729,i hak to get imposr ever tim,5
88161,i like the update but i just hate the fact tha...,2
33126,would love to give this star a it a great game...,2
74405,wow this game really change i miss the old one...,2


2    5358
4    5358
3    5358
5    5358
1    5358
Name: score, dtype: int64

In [88]:
x = data['content']
y = data['score']

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
one_hot = encoder.fit_transform(y.values.reshape(-1, 1))
y = one_hot.toarray()

x.shape, y.shape

((26790,), (26790, 5))

In [89]:
MAX_LENGTH = 100

In [90]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=0)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(21699,) (21699, 5)
(2412,) (2412, 5)
(2679,) (2679, 5)


In [91]:
# w2v = Word2Vec(
#     [word_tokenize(sentence.lower()) for sentence in x_train.values],
#     vector_size=MAX_LENGTH,
#     min_count=1
# )

In [92]:
# len(w2v.wv.index_to_key)

In [93]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_val_sequences = tokenizer.texts_to_sequences(x_val)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

for seq in x_train_sequences[:3]:
    print(seq)

[35, 87, 25, 5808, 1107, 5809]
[2, 49, 17, 9, 5, 143, 3, 167, 364, 65, 57, 37, 234, 3, 92, 206, 25, 6, 543, 2, 195, 1920, 3, 47, 6, 19, 76, 48, 9, 384, 16, 119, 384, 4, 198, 57, 34, 24, 2, 107, 7, 27, 33, 99, 124, 9, 5, 125, 7, 40, 33, 38, 66, 17, 9, 60, 23, 27, 33, 99, 35, 63, 188, 2, 49, 72, 7, 15, 9, 5, 91]
[89, 452, 89, 26, 65, 153, 6, 102, 89, 63, 512, 231, 13, 3, 2, 297, 17, 34, 15, 207, 5, 673]


In [94]:
x_train = pad_sequences(x_train_sequences, 
                                maxlen=MAX_LENGTH,
                                padding='post',
                                truncating='post')

x_val = pad_sequences(x_val_sequences,
                             maxlen=MAX_LENGTH,
                             padding='post',
                             truncating='post')

x_test = pad_sequences(x_test_sequences,
                             maxlen=MAX_LENGTH,
                             padding='post',
                             truncating='post')

for seq in x_train[:3]:
    print(seq)

[  35   87   25 5808 1107 5809    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[   2   49   17    9    5  143    3  167  364   65   57   37  234    3
   92  206   25    6  543    2  195 1920    3   47    6   19   76   48
    9  384   16  119  384    4  198   57   34   24    2  107    7   27
   33   99  124    9    5  125    7   40   33   38   66   17    9   60
   23   27   33   99   35   63  188    2   49   72    7   15    9    5
   91    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0

In [95]:


embeddings_dict = {}
with open('glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], 'float32')
        embeddings_dict[word] = vector
len(embeddings_dict)

400000

In [96]:
embedding_matrix = np.zeros((len(word_index)+1, MAX_LENGTH))
for word,i in word_index.items():
    embedding_matrix[i] = embeddings_dict.get(word, np.zeros(MAX_LENGTH))
    
embedding_matrix.shape

(16233, 100)

In [97]:
# model = Sequential([
#     Embedding(len(word_index)+1, MAX_LENGTH, input_length=MAX_LENGTH, weights=[embedding_matrix]),
#     GRU(128,return_sequence=True),
#     GRU(128,return_sequence=True),
#     GRU(128),
#     Dense(128, activation='relu'),
#     Dense(5, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [98]:
x_train

array([[ 35,  87,  25, ...,   0,   0,   0],
       [  2,  49,  17, ...,   0,   0,   0],
       [ 89, 452,  89, ...,   0,   0,   0],
       ...,
       [  2,  17,   9, ...,   0,   0,   0],
       [776,  51, 459, ...,   0,   0,   0],
       [  2, 294,   4, ...,   0,   0,   0]])

In [99]:
baseline_model = Sequential()
baseline_model.add(Embedding(len(word_index)+1, MAX_LENGTH, input_length=MAX_LENGTH, weights=[embedding_matrix]))
baseline_model.add(GRU(32, return_sequences=True))
baseline_model.add(GRU(32, return_sequences=True))
baseline_model.add(GRU(32))
baseline_model.add(Dense(5, activation='softmax'))
baseline_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
baseline_model.fit(x_train, y_train, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
679/679 [==============================] - 72s 98ms/step - loss: 1.6088 - accuracy: 0.2028 - val_loss: 1.6068 - val_accuracy: 0.2226
Epoch 2/20
679/679 [==============================] - 65s 95ms/step - loss: 1.5211 - accuracy: 0.2978 - val_loss: 1.3274 - val_accuracy: 0.4229
Epoch 3/20
679/679 [==============================] - 65s 96ms/step - loss: 1.2910 - accuracy: 0.4466 - val_loss: 1.2512 - val_accuracy: 0.4639
Epoch 4/20
679/679 [==============================] - 66s 97ms/step - loss: 1.1916 - accuracy: 0.5027 - val_loss: 1.2783 - val_accuracy: 0.4590
Epoch 5/20
 59/679 [=>............................] - ETA: 58s - loss: 1.0699 - accuracy: 0.5588